In [1]:
#%pip install -q wandb
import wandb
wandb.login()

#from google.colab import drive
#drive.mount('/content/drive')

wandb: Currently logged in as: woodleighj (jackwoodleigh). Use `wandb login --relogin` to force relogin


True

In [2]:
#%pip install -q einops
import os
import torch
import torchvision
import torchvision.transforms as transforms
from Diffusion import DiffusionModel
from utils import *

In [3]:
img_size = 32
batch_size = 32
epochs = 100
learning_rate = 0.0001
min_learning_rate= 0.00001
log = True
noise_steps = 1000
noise_schedule = "linear"
block_structure = [1, 1, 1]
block_multiplier = [2, 2, 2]
d_model = 64

In [4]:
transform = transforms.Compose([
            transforms.Resize(32),
            #transforms.RandomResizedCrop(img_size, scale=(0.8, 1.0)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # normalize mean and std for n=3 color channels
        ])

In [5]:
training_set = torchvision.datasets.CIFAR10('./data', train=True, transform=transform, download=True)
testing_set = torchvision.datasets.CIFAR10('./data', train=False, transform=transform, download=True)
training_set, validation_set = torch.utils.data.random_split(training_set, [45000, 5000])

Files already downloaded and verified
Files already downloaded and verified


In [6]:
training_loader = torch.utils.data.DataLoader(training_set, batch_size=batch_size, shuffle=True, num_workers=10,  pin_memory=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=batch_size, shuffle=True, num_workers=10, pin_memory=True)
testing_loader = torch.utils.data.DataLoader(testing_set, batch_size=batch_size, shuffle=True, num_workers=10, pin_memory=True)

In [7]:
model = DiffusionModel(
    d_model=d_model,
    block_structure=block_structure,
    block_multiplier=block_multiplier,
    learning_rate=learning_rate,
    min_learning_rate=min_learning_rate,
    in_channels=3,
    out_channels=3,
    noise_steps=noise_steps,
    beta_start=1e-4,
    beta_end=0.02,
    img_size=img_size,
    noise_schedule=noise_schedule,
    device="cuda"
)

In [8]:
model.print_parameter_count()

47610947


In [12]:
path = './saves/model_save.pt'

In [ ]:
if log:
    wandb.init(
        project="DenoiseDiffusion",
        config={
        "dataset": "CIFAR-100",
        "img_size": img_size,
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "epochs": epochs,
        "noise_schedule": noise_schedule,
        "d_model": d_model,
        "block_structure": block_structure,
        "block_multiplier": block_multiplier,
        "Type": "DDPM_V2",
        "Save": "Save-0"
        }
    )

model.train_model(validation_loader, validation_loader, epochs, log=log, save_path=path)
wandb.finish()

Training_Loss,▁
Validation_Loss,▁
Training_Loss,0.06197
Validation_Loss,0.04922


Epoch 0...


100%|████████████████████████████████████████████████████████████████████████████████| 157/157 [00:24<00:00,  6.28it/s]


Training Loss: 0.05219194635891231, Validation Loss: 0.04901714235022189
